In [14]:
import pandas as pd
import numpy as np


In [15]:
rawdf = pd.read_csv('GreenspaceDownload/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.csv', encoding='unicode_escape', low_memory=False)

In [16]:
rawdf.shape

(36902, 160)

In [17]:
print(list(rawdf.columns))

['ID_HDC_G0', 'QA2_1V', 'AREA', 'BBX_LATMN', 'BBX_LONMN', 'BBX_LATMX', 'BBX_LONMX', 'GCPNT_LAT', 'GCPNT_LON', 'CTR_MN_NM', 'CTR_MN_ISO', 'XBRDR', 'XCTR_NBR', 'XC_NM_LST', 'XC_ISO_LST', 'GRGN_L1', 'GRGN_L2', 'UC_NM_MN', 'UC_NM_LST', 'UC_NM_SRC', 'H75_NBR', 'H90_NBR', 'H00_NBR', 'H75_AREA', 'H90_AREA', 'H00_AREA', 'E_BM_NM_LST', 'E_SL_LST', 'EL_AV_ALS', 'E_KG_NM_LST', 'E_RB_NM_LST', 'E_WR_P_90', 'E_WR_P_00', 'E_WR_P_14', 'E_WR_T_90', 'E_WR_T_00', 'E_WR_T_14', 'B75', 'B90', 'B00', 'B15', 'P75', 'P90', 'P00', 'P15', 'BUCAP75', 'BUCAP90', 'BUCAP00', 'BUCAP15', 'NTL_AV', 'GDP90_SM', 'GDP00_SM', 'GDP15_SM', 'INCM_CMI', 'DEV_CMI', 'TT2CC', 'E_GR_AV90', 'E_GR_AV00', 'E_GR_AV14', 'E_GR_AH90', 'E_GR_AM90', 'E_GR_AL90', 'E_GR_AT90', 'E_GR_AH00', 'E_GR_AM00', 'E_GR_AL00', 'E_GR_AT00', 'E_GR_AH14', 'E_GR_AM14', 'E_GR_AL14', 'E_GR_AT14', 'E_EC2E_E75', 'E_EC2E_E90', 'E_EC2E_E00', 'E_EC2E_E15', 'E_EC2E_R75', 'E_EC2E_R90', 'E_EC2E_R00', 'E_EC2E_R15', 'E_EC2E_I75', 'E_EC2E_I90', 'E_EC2E_I00', 'E_EC2E_I15

In [39]:
cols_to_keep = ['GCPNT_LAT', 'GCPNT_LON', 'CTR_MN_NM', 'UC_NM_MN', 'UC_NM_LST', 'E_GR_AV14', 'E_GR_AT14', 'SDG_A2G14', 'SDG_OS15MX', 'P15', 'B15', 'BUCAP15', 'INCM_CMI', 'DEV_CMI', 'GDP15_SM', 'E_BM_NM_LST', 'E_WR_T_14']
df = rawdf[cols_to_keep]
df = df[df['CTR_MN_NM'] == 'United States']
df.replace(to_replace=['?', '??', '???', 'NAN'], value = [np.nan, np.nan, np.nan, np.nan], inplace=True)
#df.head(40)
a = df.duplicated(subset='UC_NM_MN')
dupind = list(a.index[a])
print(dupind)
duplicates = df[df.index.isin(dupind)]
duplicates

[524, 626, 649, 728, 876, 909, 1003, 1028, 1044]


,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,UC_NM_MN,UC_NM_LST,E_GR_AV14,E_GR_AT14,SDG_A2G14,SDG_OS15MX,P15,B15,BUCAP15,INCM_CMI,DEV_CMI,GDP15_SM,E_BM_NM_LST,E_WR_T_14
524,39.786790,-89.653699,United States,Springfield,Springfield,0.55749533,110.851377,0.860306,89.06,1.096793e+05,80.396439,733.013676,HIC,MDR,3.588679e+09,"Temperate Grasslands, Savannas, and Shrublands",12.48464513
626,39.159467,-86.534890,United States,Bloomington,Bloomington,0.548406735,44.919931,0.715186,70.73,7.442638e+04,28.322845,380.548464,HIC,MDR,1.652635e+09,Temperate Broadleaf and Mixed Forests,12.32019329
649,40.416010,-86.876922,United States,Lafayette,Lafayette; West Lafayette,0.493745658,80.201392,0.579613,67.16,1.161373e+05,54.112743,465.937498,HIC,MDR,3.249214e+09,Temperate Broadleaf and Mixed Forests,11.39722514
728,40.014787,-82.999739,United States,Columbus,Columbus,0.514367151,846.410100,0.726532,34.78,1.138151e+06,553.072327,485.939120,HIC,MDR,3.430752e+10,Temperate Broadleaf and Mixed Forests,10.69221854
876,39.200600,-76.821448,United States,Columbia,Columbia,0.547956846,74.504696,0.903598,81.17,1.056193e+05,37.420444,354.295383,HIC,MDR,4.895044e+09,Temperate Broadleaf and Mixed Forests,13.17422318
909,43.172372,-77.630233,United States,Rochester,Rochester,0.553536349,256.502786,0.827774,79.07,3.883153e+05,179.111542,461.252916,HIC,MDR,1.519912e+10,Temperate Broadleaf and Mixed Forests,9.697564363
1003,42.146355,-72.568555,United States,Springfield,Springfield,0.499681101,159.701848,0.534996,37.64,2.529464e+05,99.783127,394.483313,HIC,MDR,1.041693e+10,Temperate Broadleaf and Mixed Forests,9.893814564
1028,42.709273,-71.163612,United States,Lawrence,Lawrence; Methuen; Andover; North Andover; Nor...,0.519911179,44.389289,0.500628,75.79,1.159924e+05,30.434315,262.382010,HIC,MDR,4.984015e+09,Temperate Broadleaf and Mixed Forests,9.908699036
1044,43.657455,-70.292096,United States,Portland,Portland,0.530716868,83.135018,0.726120,75.47,8.873563e+04,42.298477,476.679749,HIC,MDR,2.344911e+09,Temperate Broadleaf and Mixed Forests,8.630513191


In [40]:
temp = df[df['UC_NM_MN'] == "Portland"]
temp

,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,UC_NM_MN,UC_NM_LST,E_GR_AV14,E_GR_AT14,SDG_A2G14,SDG_OS15MX,P15,B15,BUCAP15,INCM_CMI,DEV_CMI,GDP15_SM,E_BM_NM_LST,E_WR_T_14
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286
1044,43.657455,-70.292096,United States,Portland,Portland,0.530716868,83.135018,0.726120,75.47,8.873563e+04,42.298477,476.679749,HIC,MDR,2.344911e+09,Temperate Broadleaf and Mixed Forests,8.630513191


Convert ??? and NAN to NaN, filter to United States